In [1]:
from datasets import load_dataset
from PIL import Image
import os
from functools import partial

# load json file
json_data_path = "/home/linux/yyj/colpali/finetune/split_2x2_image_dense_cap_pairs_t.json"
ds = load_dataset("json", data_files=json_data_path, split="train")

# load images
image_dir = "/home/linux/yyj/colpali/finetune/pdfTo_2x2_image_dense"
def load_image(example, image_dir):
    full_path = os.path.join(image_dir, example["image_path"])
    example["image"] = Image.open(full_path)
    return example

ds = ds.map(partial(load_image, image_dir=image_dir))

# split dataset
ds = ds.train_test_split(test_size=0.2, seed=42)
train_ds = ds["train"]
test_ds = ds["test"]


/home/linux/anaconda3/envs/colpali/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
from transformers import Trainer, TrainingArguments, BitsAndBytesConfig, \
    ColPaliForRetrieval, ColPaliProcessor, EarlyStoppingCallback
from colpali_engine.loss import ColbertPairwiseCELoss
from colpali_engine.trainer.contrastive_trainer import ContrastiveTrainer
from peft import LoraConfig, get_peft_model

torch.manual_seed(42)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_dir = "/home/linux/yyj/colpali/finetune/colpali-v1.2-hf"
model = ColPaliForRetrieval.from_pretrained(
    model_dir,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
    device_map="cuda:0",
).eval()


lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
        init_lora_weights="gaussian"
    )
lora_config.inference_mode = False
model = get_peft_model(model, lora_config)
processor = ColPaliProcessor.from_pretrained(model_dir)

def collate_fn(examples):
    texts = []
    images = []

    for example in examples:

        texts.append(example["caption"])
        images.append(example["image"].convert("RGB"))

    batch_images = processor(images=images, return_tensors="pt").to(model.device)
    batch_queries = processor(text=texts, max_length=80, padding="max_length", truncation=True, return_tensors="pt").to(model.device)
    return (batch_queries, batch_images)


class ContrastiveTrainer(Trainer):
    def __init__(self, loss_func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.loss_func = loss_func

    def compute_loss(self, model, inputs, num_items_in_batch=4, return_outputs=False):
        query_inputs, doc_inputs = inputs
        query_outputs = model(**query_inputs)
        doc_outputs = model(**doc_inputs)
        loss = self.loss_func(query_outputs.embeddings, doc_outputs.embeddings)
        return (loss, (query_outputs, doc_outputs)) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only=True, ignore_keys=None):
        query_inputs, doc_inputs = inputs # unpack from data collator
        with torch.no_grad():
            query_outputs = model(**query_inputs)
            doc_outputs = model(**doc_inputs)

            loss = self.loss_func(query_outputs.embeddings, doc_outputs.embeddings)
            
            return loss, None, None if prediction_loss_only else loss

training_args = TrainingArguments(
    output_dir="./colpali_city_0605",
    num_train_epochs=1.5,
    per_device_train_batch_size=5,
    gradient_accumulation_steps=4,
    gradient_checkpointing=False,
    logging_steps=10,
    eval_strategy="steps",    
    eval_steps=10,
    warmup_steps=20,
    learning_rate=5e-5,
    save_total_limit=1,
    report_to="tensorboard",
    dataloader_pin_memory=False,
    load_best_model_at_end=True,      
    metric_for_best_model="loss",      
    greater_is_better=False   
)


trainer = ContrastiveTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    args=training_args,
    loss_func=ColbertPairwiseCELoss(),
    data_collator=collate_fn,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.args.remove_unused_columns = False


pynvml not found. GPU stats will not be printed.


Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.87s/it]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [3]:
trainer.train()

Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss
10,0.462400,0.144549
20,0.299500,0.089222
30,0.151200,0.055219
40,0.171500,0.041434
50,0.168000,0.039521
60,0.109400,0.037996
70,0.097300,0.031495
80,0.057200,0.026469
90,0.047700,0.027724
100,0.036800,0.026447


TrainOutput(global_step=220, training_loss=0.09791123878888108, metrics={'train_runtime': 3022.183, 'train_samples_per_second': 1.563, 'train_steps_per_second': 0.078, 'total_flos': 0.0, 'train_loss': 0.09791123878888108, 'epoch': 1.4})

In [4]:
trainer.save_model("/home/linux/yyj/colpali/finetune/wiky_city_zh_t_0605")  # 包括模型和 tokenizer 等信息
processor.save_pretrained("/home/linux/yyj/colpali/finetune/wiky_city_zh_t_0605")  # 保存预处理器，如 tokenizer + image processor


[]